# Gridding micromagnetic simulations

In [2]:
import pandas
import numpy as np
import scipy.interpolate#
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import copy

%matplotlib qt

## Data input

- Import .dat file with simulated x, y, z, mx, my, mz 

In [3]:
#folder = r'/home/grl31/Dropbox (Cambridge University)/Linux-Surface share/Magnetic simulations/size_hyst_ring3'
folder = r'C:/Users/grl31/Dropbox (Cambridge University)/Linux-Surface share/Magnetic simulations/size_hyst_ring3'
fname = 'ring3_down.70nm.dat' # blue
#fname = 'ring2_down.60nm.dat' # red
#fname = 'ring1_down.80nm.dat' # green
fpath = folder + '/' + fname

# 'fixed width formatted' works directly with even .dat format
data = pandas.read_fwf(fpath, header = None)

all_x = np.asarray(data[0])
all_y = np.asarray(data[1])
all_z = np.asarray(data[2])

m_x = np.asarray(data[3])
m_y = np.asarray(data[4])
m_z = np.asarray(data[5])

# format to be in an (n,D) array
coor = np.array([all_x,all_y,all_z]).T
ms = np.array([m_x,m_y,m_z]).T

In [5]:
np.shape(m_x)

(5273,)

- Generate desired grid (note it should be roughly the same coordinate range)

In [32]:
# find boundaries of the dataset
xi = np.floor(np.min(all_x))
xf = np.ceil(np.max(all_x))
yi = np.floor(np.min(all_y))
yf = np.ceil(np.max(all_y))
zi = np.floor(np.min(all_z))
zf = np.ceil(np.max(all_z))

# mgrid forms a grid by 'indexing' what you want
# note that indexing format is start:stop:stepj
grid_x, grid_y, grid_z = np.mgrid[xi:xf:100j, yi:yf:100j, zi:zf:100j]
coor_grid = np.array([grid_x, grid_y, grid_z]).T

print('x:', xi,xf, 'y:', yi,yf, 'z:', zi,zf)

x: 12.0 15.0 y: 23.0 26.0 z: 0.0 2.0


- Scale grid to the right size in nm

In [20]:
# xsize = (xf - xi) * 1000
# ysize = (yf - yi) * 1000
# zsize = (zf - zi) * 1000

# factor = 70 / 2500 # blue ring 3

# grid_x = grid_x - xi
# grid_x = grid_x * 1000 * factor

# grid_y = grid_y - yi
# grid_y = grid_y * 1000 * factor

# grid_z = grid_z - zi
# grid_z = grid_z * 1000 * factor

# xi, xf = np.min(grid_x), np.max(grid_x)
# yi, yf = np.min(grid_y), np.max(grid_y)
# zi, zf = np.min(grid_z), np.max(grid_z)

# print('x:', xi,xf, 'y:', yi,yf, 'z:', zi,zf)
# coor_grid = np.array([grid_x, grid_y, grid_z]).T
# coor = coor*1000*factor

x: 0.0 84.0 y: 0.0 84.0 z: 0.0 56.0


## Interpolate

In [71]:
# note that mx, my and mz must be done separately
output_x = scipy.interpolate.griddata(coor,m_x,(grid_x,grid_y,grid_z),method='linear',fill_value=0)
output_y = scipy.interpolate.griddata(coor,m_y,(grid_x,grid_y,grid_z),method='linear',fill_value=0)
output_z = scipy.interpolate.griddata(coor,m_z,(grid_x,grid_y,grid_z),method='linear',fill_value=0)

In [72]:
coor.shape, m_x.shape, grid_x.shape

((5273, 3), (5273,), (100, 100, 100))

In [77]:
plt.imshow(output_x[:,:,40], cmap = 'RdBu_r')

## Mask out hole

- For each point in the gridded data, find out how far away it is from one of the original input data points 

In [35]:
tree = scipy.spatial.cKDTree(coor)
d, inds = tree.query(coor_grid, k=1, p=2)

- Visually inspect and decide what the threshold distance should be

In [64]:
%matplotlib qt
plt.imshow(np.flipud(d[40,:,:]), cmap = 'Blues_r', extent=[xi,xf,yi,yf])
plt.axis('off')

(12.0, 15.0, 23.0, 26.0)

- create gridded data mask which is 0 for regions above threshold distance and 1 for those within

In [65]:
threshold = 0.08
i_out = d > threshold
i_in = d <= threshold
mask = copy.copy(d)
mask[i_out] = 0
mask[i_in] = 1

plt.imshow(np.flipud(mask[40,:,:]))
plt.axis('off')

(-0.5, 99.5, 99.5, -0.5)

- apply mask to output data

In [45]:
output_x = output_x * mask.T
output_y = output_y * mask.T
output_z = output_z * mask.T

## Visualise

- view xy projection and compare to input data for Mx

In [60]:
# Input data
plt.scatter(all_x,all_y,color='grey', vmin=-2, vmax= 2,alpha=0.2,marker='.')

# Interpolated values
mx_proj = np.average(output_x,axis=2)
plt.imshow(np.flipud((mx_proj.T**2 + my_proj.T**2)**0.5), cmap='RdBu', vmin=-1.2, vmax=1.2, extent=[xi,xf,yi,yf])
plt.axis('off')

(12.0, 15.0, 23.0, 26.0)

- view xy projection of magnetisation as a vector

In [63]:
# average over the z axis for gridded coordinates and gridded magnetisations
x_proj = np.average(grid_x,axis=2)
y_proj = np.average(grid_y,axis=2)

mx_proj = np.average(output_x,axis=2)
my_proj= np.average(output_y,axis=2)

# Plot magnitude of M_proj in the background
M_proj = np.hypot(mx_proj,my_proj)
plt.imshow(np.flipud(M_proj.T), extent=[xi,xf,yi,yf], cmap ='Blues', vmin=0,vmax=1)

# Plot M_proj as vectors
s = 5
plt.quiver(x_proj[::s,::s], y_proj[::s,::s], mx_proj[::s,::s], my_proj[::s,::s])
plt.axis('off')

(12.0, 15.0, 23.0, 26.0)

- View 3D plot of gridded magnetisation

In [142]:
fig = plt.figure(figsize=(8,8))
ax = fig.gca(projection='3d')
s = 5

# View gridded data
a, b, c, d, e, f = grid_x,grid_y,grid_z,output_x,output_y,output_z
ax.quiver(a[::s,::s,::s], b[::s,::s,::s], c[::s,::s,::s], d[::s,::s,::s], e[::s,::s,::s], f[::s,::s,::s],length = .3,alpha=0.5,linewidth=3,color='k')

# View original input data
# a, b, c, d, e, f = all_x,all_y,all_z,m_x,m_y,m_z
# ax.quiver(a[::s], b[::s], c[::s], d[::s], e[::s], f[::s],length = 0.1,alpha=0.5,linewidth=3,color='k')

plt.xlabel('x', fontsize=20)
plt.ylabel('y', fontsize=20)
ax.set_zlabel('z', fontsize=20)

Text(0.5, 0, 'z')

## Save gridded data

In [153]:
xsize = (xf - xi) * 1000
ysize = (yf - yi) * 1000
zsize = (zf - zi) * 1000
factor = 70 / 2500 # blue ring 3
xsize*factor, ysize*factor, zsize*factor

(112.0, 84.0, 56.0)

In [141]:
grid_x = grid_x - xi
grid_x = grid_x * 1000 * factor

grid_y = grid_y - yi
grid_y = grid_y * 1000 * factor

grid_z = grid_z - zi
grid_z = grid_z * 1000 * factor

In [65]:
save_gx = grid_x.reshape(100**3)
save_gy = grid_y.reshape(100**3)
save_gz = grid_z.reshape(100**3)
save_mx = output_x.reshape(100**3)
save_my = output_y.reshape(100**3)
save_mz = output_z.reshape(100**3)

save_data = np.array([save_gx,save_gy,save_gz,save_mx,save_my,save_mz]).T

In [71]:
save_folder = r'/home/grl31/Dropbox (Cambridge University)/Linux-Surface share/Magnetic simulations'
save_name = r'/gridded_ring3_70nm_blue.csv'
save_fpath = save_folder + save_name

In [59]:
np.savetxt(save_fpath,save_data)

In [54]:
save_data.shape

(6, 1000000)

In [58]:
aaa = np.linspace(0,100,101)
bbb = aaa + 100
ccc = bbb + 100
sss = np.array([aaa,bbb,ccc])
save_fpath = save_folder + r'/testsave2.csv'

AttributeError: module 'pandas' has no attribute 'load_txt'

In [72]:
df = pandas.DataFrame(data=save_data)

In [74]:
pandas.DataFrame.to_csv(df,save_fpath,index=False,header=False)